In [113]:
import pandas as pd
import numpy as np
from spacy.en import English
nlp = English()

In [114]:
sql_select = '''
select created_at, location, tweet_content, cleaned_tweet, vector from tweets;
'''

import psycopg2 as pg2
import psycopg2.extras as pgex
this_host='34.211.59.66'
this_user='postgres'
this_password='postgres'
conn = pg2.connect(host = this_host, 
                        user = this_user,
                        password = this_password)
cur = conn.cursor(cursor_factory=pgex.RealDictCursor)
cur.execute(sql_select)
rows = cur.fetchall()
conn.close()
df = pd.DataFrame(rows)

In [115]:
df['created_datetime'] = pd.to_datetime(df['created_at'])
df['year'] = df.created_datetime.apply(lambda x: x.year)
df['month'] = df.created_datetime.apply(lambda x: x.month)
df['day'] = df.created_datetime.apply(lambda x: x.day)
df['dayofweek'] = df.created_datetime.apply(lambda x: x.dayofweek)
df['hour'] = df.created_datetime.apply(lambda x: x.hour)

In [116]:
len(df)

390117

In [117]:
Paris_accords_df = df[(df['created_datetime'] > '2017-06-01 14:35:00')][(df['created_datetime'] < '2017-06-03 14:35:00')]
Paris_accords_df['tweet_content'].sample(10).values

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


array(['exit 11b',
       'this joint  #losfeliz #hidden #bar #losangeles #hollywood #birthday @ tenants of the trees https://t.co/ni3nzvu754',
       'i have the best glam team around 😻🔥 https://t.co/f3jvqdyfgw',
       'i feel attacked @shmasonayala',
       'ok but when he was up in the majors he was terrible https://t.co/iw1hhx4vr6',
       'my prayers have been answered. #thanksgod #pokemonsunmoon #pokemon #harambe https://t.co/ymyrxhseap',
       "i look like i'm balding. #wtf? shiiiet... still handsome tho. #ftw #podcast @itsdpd #skincare… https://t.co/ajjx26dhwq",
       "@gameofthrones it's almost fucking summer!",
       "me at oomf cause ew i can't believe i... https://t.co/prd6xnl6at",
       '@venicemase  just checking to see if your good after eating 13 donuts in 5 minutes.  #tde the doubt eater'], dtype=object)

In [118]:
paris_accords = Paris_accords_df[Paris_accords_df['tweet_content'].str.contains(('paris|climate'))]
paris_accords.reset_index(inplace = True)
paris_accords['cleaned_tweet'].sample(1).values

array([ 'never have i supported the united nations carbon climate agenda nor other banking aspects of the climate agenda.'], dtype=object)

In [119]:
import random
A_index = list(range(paris_accords.shape[0]))
random.shuffle(A_index)

In [120]:
n = 200
A1_index = []
for i in range(n):
    A1_index.append(A_index.pop())

In [121]:
A1 = paris_accords.iloc[A1_index,:]
A2 = paris_accords.iloc[A_index,:]

In [122]:
A2.shape

(2414, 12)

In [123]:
A1.shape

(200, 12)

In [124]:
A1.reset_index(inplace = True)
A1_vec = np.array([nlp(i).vector for i in A1['cleaned_tweet']])
A1_vec.shape

(200, 300)

In [125]:
A2.reset_index(inplace = True)
A2_vec = np.array([nlp(i).vector for i in A2['cleaned_tweet']])
A2_vec.shape

(2414, 300)

In [126]:
non_paris_accords = Paris_accords_df[~Paris_accords_df['cleaned_tweet'].str.contains(('paris|climate|accord|trump'))]
non_paris_accords.reset_index(inplace = True)

In [127]:
B_index = list(range(non_paris_accords.shape[0]))
random.shuffle(B_index)

In [128]:
n = 200
B1_index = []
for i in range(n):
    B1_index.append(B_index.pop())
m = A2.shape[0]
B2_index = []
for i in range(m):
    B2_index.append(B_index.pop())    

In [129]:
B1 = non_paris_accords.iloc[B1_index,:]
B2 = non_paris_accords.iloc[B2_index,:]

In [130]:
B1.reset_index(inplace = True)
B1_vec = np.array([nlp(i).vector for i in B1['cleaned_tweet']])
B1_vec.shape

(200, 300)

In [131]:
B2.reset_index(inplace = True)
B2_vec = np.array([nlp(i).vector for i in B2['cleaned_tweet']])
B2_vec.shape

(2414, 300)

In [132]:
A1_vec_mean = np.average(A1_vec, axis=0)
A2_vec_mean = np.average(A2_vec, axis=0)
B1_vec_mean = np.average(B1_vec, axis=0)
B2_vec_mean = np.average(B2_vec, axis=0)

In [133]:
from sklearn.metrics.pairwise import cosine_similarity

In [134]:
cosine_similarity(A1_vec_mean.reshape(1,-1),A2_vec_mean.reshape(1,-1))

array([[ 0.99916667]], dtype=float32)

In [135]:
cosine_similarity(B2_vec_mean.reshape(1,-1),B1_vec_mean.reshape(1,-1))

array([[ 0.99919271]], dtype=float32)

In [136]:
cosine_similarity(A1_vec_mean.reshape(1,-1),B1_vec_mean.reshape(1,-1))

array([[ 0.97586632]], dtype=float32)

In [137]:
cosine_similarity(A2_vec_mean.reshape(1,-1),B2_vec_mean.reshape(1,-1))

array([[ 0.97923732]], dtype=float32)

In [ ]:
def Similarity_score(text):
    wiki_cat_df = pd.DataFrame(list(cli.wiki_mongo_database.wiki_cat_collection.find({})))
    #cat_vecs = np.array([nlp(i).vector for i in wiki_cat_df['text']])
    r = redis.StrictRedis(redis_ip)
    cat_vecs = pickle.loads(r.get('nlp_cat_vecs'))
    similarity_score={}
    for j in range(len(cat_vecs)):
        similarity = cosine_similarity(cat_vecs[j].reshape(1,-1),nlp(text).vector.reshape(1,-1))
        similarity_score[(wiki_cat_df['category'][j])] = round(similarity[0][0],3)
        cs_s_df = pd.DataFrame.from_dict(similarity_score,orient='index')
        cs_s_df.columns = ['score']
        cs_s_df = cs_s_df.sort_values('score',ascending=False)
    return cs_s_df.head(3)

In [ ]:
sql_select = '''
select id, tweet_content, count(*) from tweets group by id, tweet_content HAVING count(*) > 1 order by id
'''

import psycopg2 as pg2
import psycopg2.extras as pgex
this_host='34.211.59.66'
this_user='postgres'
this_password='postgres'
conn = pg2.connect(host = this_host, 
                        user = this_user,
                        password = this_password)
cur = conn.cursor(cursor_factory=pgex.RealDictCursor)
cur.execute(sql_select)
rows = cur.fetchall()
conn.close()
df = pd.DataFrame(rows)

In [ ]:
np.array(df['id'])

In [ ]:
sql_select = '''
select id, tweet_content from tweets where id in 
        ('870369089822236672', '870369123766685698', '870412757312647168',
       '870736125140156416', '870736377163333632', '870736526442876929',
       '870736651051442176', '870736815195475968', '870736826973036544',
       '870737064672636928', '870737118456201216', '870737249171685376',
       '870737290254929920', '870737449386876928', '870737688697196545',
       '870737824277962752', '870738170819641345', '870739370730377216',
       '870739784519393280', '870740002203881472', '870740231019937792',
       '870740967396999168', '870740985059303424', '870741079565324288',
       '870741317936062464', '870826652044939264', '871071496130646016',
       '871107396843614208', '871414307308789760', '871865745357058049') order by id
'''

import psycopg2 as pg2
import psycopg2.extras as pgex
this_host='34.211.59.66'
this_user='postgres'
this_password='postgres'
conn = pg2.connect(host = this_host, 
                        user = this_user,
                        password = this_password)
cur = conn.cursor(cursor_factory=pgex.RealDictCursor)
cur.execute(sql_select)
rows = cur.fetchall()
conn.close()
df = pd.DataFrame(rows)

In [ ]:
sql_delete = '''
alter table tweets 
alter column id set not null,
ADD PRIMARY KEY (id);   
'''
alter column id type integer,
import psycopg2 as pg2
import psycopg2.extras as pgex
this_host='34.211.59.66'
this_user='postgres'
this_password='postgres'
conn = pg2.connect(host = this_host, 
                        user = this_user,
                        password = this_password)
cur = conn.cursor(cursor_factory=pgex.RealDictCursor)
cur.execute(sql_delete)
conn.commit()
#rows = cur.fetchall()
conn.close()


In [ ]:
sql = '''
BEGIN;
CREATE INDEX ON tweets (id); 
COMMIT;
'''
import psycopg2 as pg2
import psycopg2.extras as pgex
this_host='34.211.59.66'
this_user='postgres'
this_password='postgres'
conn = pg2.connect(host = this_host, 
                        user = this_user,
                        password = this_password)
cur = conn.cursor(cursor_factory=pgex.RealDictCursor)
cur.execute(sql)
conn.commit()
#rows = cur.fetchall()
conn.close()
